In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
# print(check_output(["ls", "data"]).decode("utf8"))

# Any results you write to the current directory are saved as output.


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.cluster import DBSCAN
from pprint import pprint
# from geopy.distance import great_circle
# from shapely.geometry import MultiPoint

/Users/vipul/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def add_clusters_to_df(df):
    coords = df.as_matrix(columns=['latitude', 'longitude'])
    kms_per_radian = 6371.0088
    epsilon = 0.3 / kms_per_radian
    db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    cluster_labels = db.labels_
    num_clusters = len(set(cluster_labels))
    clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
    print('Number of clusters: {}'.format(num_clusters))
    
    clus = pd.Series(db.labels_.tolist())
    df['cluster'] = clus.values
    return df

In [3]:
def remove_outliers(df):
    # standard deviation threshold
    sd_threshold = 1
    
    # remove price outliers
    df = df[(df.price <= 15000) & (df.price >= 1000)]
    return df

In [4]:
df = pd.read_json(open("data/train.json", "r"))
print(df.shape)
df.describe()

(49352, 15)


,bathrooms,bedrooms,latitude,listing_id,longitude,price
count,49352.00000,49352.000000,49352.000000,4.935200e+04,49352.000000,4.935200e+04
mean,1.21218,1.541640,40.741545,7.024055e+06,-73.955716,3.830174e+03
std,0.50142,1.115018,0.638535,1.262746e+05,1.177912,2.206687e+04
min,0.00000,0.000000,0.000000,6.811957e+06,-118.271000,4.300000e+01
25%,1.00000,1.000000,40.728300,6.915888e+06,-73.991700,2.500000e+03
50%,1.00000,1.000000,40.751800,7.021070e+06,-73.977900,3.150000e+03
75%,1.00000,2.000000,40.774300,7.128733e+06,-73.954800,4.100000e+03
max,10.00000,8.000000,44.883500,7.753784e+06,0.000000,4.490000e+06


In [5]:
df = remove_outliers(df)
print(df.shape)
# df.head()
df.to_csv('t.csv')

(49077, 15)


## Feature Engineering

In [7]:
df["num_photos"] = df["photos"].apply(len)
df["num_features"] = df["features"].apply(len)
df["num_description_words"] = df["description"].apply(lambda x: len(x.split(" ")))

# Create date month year
df["created"] = pd.to_datetime(df["created"])
df["created_year"] = df["created"].dt.year
df["created_month"] = df["created"].dt.month
df["created_day"] = df["created"].dt.day
df['is_weekday'] = ((df.created_day) // 5 == 1).astype(float)

# df = add_clusters_to_df(df)
# df['cluster'] = pd.Categorical(df['cluster'])
# dfDummies = pd.get_dummies(df['cluster'], prefix = 'cluster')
# df = pd.concat([df, dfDummies], axis=1)
# df.info()

num_feats = ["bathrooms", "bedrooms", "price","latitude", "longitude",
             "num_photos", "num_features", "num_description_words",
             "created_year", "created_month", "is_weekday"]

# cluster_categories = []
# for i in range(393):
#     cluster_categories.append('cluster_' + str(i))

# num_feats = ["bathrooms", "bedrooms", "latitude", "longitude", "price",
#              "num_photos", "num_features", "num_description_words",
#              "created_year", "created_month", "created_day"]

In [8]:
# num_feats.extend(cluster_categories)
X = df[num_feats]
y = df["interest_level"]
X.head()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)

clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train, y_train)
y_val_pred = clf.predict_proba(X_val)

# calculate training loss
loss = log_loss(y_val, y_val_pred)
print(f'Loss : {loss}')

Loss : 0.6334964289689935


## Making predictions on test data

In [10]:
df = pd.read_json(open("data/test.json", "r"))
df["num_photos"] = df["photos"].apply(len)
df["num_features"] = df["features"].apply(len)
df["num_description_words"] = df["description"].apply(lambda x: len(x.split(" ")))
df["created"] = pd.to_datetime(df["created"])
df["created_year"] = df["created"].dt.year
df["created_month"] = df["created"].dt.month
df["created_day"] = df["created"].dt.day
df['is_weekday'] = ((df.created_day) // 5 == 1).astype(float)
X = df[num_feats]

y = clf.predict_proba(X)


labels2idx = {label: i for i, label in enumerate(clf.classes_)}
labels2idx


sub = pd.DataFrame()
sub["listing_id"] = df["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y[:, labels2idx[label]]
sub.to_csv("submission_rf.csv", index=False)

In [ ]:
! rm submission_rf.csv
!ls

In [ ]:
submission = pd.read_csv('submission_rf.csv')

In [9]:
submission.head()

,listing_id,high,medium,low
0,7142618,0.049,0.29600,0.65500
1,7210040,0.160,0.53700,0.30300
2,7103890,0.016,0.10500,0.87900
3,7143442,0.123,0.34975,0.52725
4,6860601,0.011,0.26700,0.72200
